In [1]:
import os
import pandas as pd

from qiskit.circuit import QuantumCircuit
import optuna
import qtpu
from qtpu.compiler.terminators import reach_num_qubits
from qtpu.compiler.success import success_num_nodes
from qtpu.compiler import trial_to_hybrid_tn

from benchmark._plot_util import *
from benchmark.util import get_hybrid_tn_info, append_to_csv, get_base_info
from benchmark.benchmarks import *

In [2]:
def get_highest_cost_trial(study: optuna.Study):
    return trial_to_hybrid_tn(
        max(study.best_trials, key=lambda t: (t.values[0], t.values[1]))
    )

In [10]:
for cost in [1e5, 1e6, 1e7]:

    circuit = brick_ansatz(100, 3)
    study = qtpu.compiler.hyper_optimize(circuit, max_cost=cost, show_progress_bar=True)
    htn = get_highest_cost_trial(study)
    info = get_hybrid_tn_info(htn)
    base_info = get_base_info(circuit)
    append_to_csv(
        "results/fidelity.csv",
        {
            "name": "brick_100_3",
            "given_cost": np.log10(cost),
            **info,
            **base_info,
        },
    )

  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
df = pd.read_csv("results/fidelity.csv")

df["rel_cx"] = df["cx_count"] / df["base_cx_count"]
df

,name,given_cost,contract_cost,bruteforce_cost,num_qpds,num_subcircuits,num_instances,esp,depth,max_qubits,swap_count,cx_count,2q_count,base_qubits,base_depth,base_esp,base_swap_count,base_cx_count,base_2q_count,rel_cx
0,linear_100_3,2.0,3.024,2.505,3,2,160,0.849,56,51,0,148,148,100,106,0.721,0,297,297,0.498316
1,linear_100_3,3.0,3.119,3.380,4,3,200,0.850,56,51,0,148,148,100,106,0.721,0,297,297,0.498316
2,linear_100_3,4.0,4.072,6.595,9,4,9360,0.906,35,33,0,90,90,100,106,0.721,0,297,297,0.303030
3,brick_100_3,2.0,3.151,2.602,3,2,200,0.847,9,52,0,151,151,100,9,0.721,0,297,297,0.508418
4,brick_100_3,3.0,3.279,2.699,3,2,250,0.850,9,50,0,147,147,100,9,0.721,0,297,297,0.494949
5,brick_100_3,4.0,3.810,5.584,7,6,4016,0.851,9,50,0,147,147,100,9,0.721,0,297,297,0.494949
